# Individual Planning Stage of Data Science 100 Project #

## DESCRIPTION OF DATA SETS: ##
### players.csv ###
This data set has 196 observations with 7 variables

| Variable Name | Type   | Meaning |
|----------------|-------------------|----------|
| experience     | Categorical (factor) | Player's Experience Level (Beginner, Amateur, Regular, Pro, Veteran|
| subscribe      | Logical (True/False)  | TRUE = Subcribed, otherwise, unsubscribed |
| hashedEmail    | chr    | Hashed Email of a Player |
| played_hours   | numeric (float)  | Number of Hours Played |
| name           | chr  | Player's Name |
| gender         | Categorical (factor)  | Geneder of Player |
| Age            | numeric (int)  | Age of Player |

### sessions.csv ###
This data set has 1535 observations with 5 variables
| Variable Name | Type   | Meaning |
|----------------|-------------------|----------|
| hashedEmail    | Character | Anonymized unique player identifier |
| start_time     | Datetime (string)  | Timestamp for when session started in DD/MM/YYYY HH:MM format|
| end_time       | Datetime (string)  | Timestamp for when session ended in DD/MM/YYYY HH:MM format |
| original_start_time | numeric (float)  | Start time in milliseconds since 01/01/1970 |
| original_end_time   | numeric (float)  | End time in milliseconds since 01/01/1970 |





## Potential Issues ##
- In players.csv, the column names are not standardized. Age is capitalized, while the other variable names are not

- In players.csv, experience could be self-reported, which might be an inaccurate representation of actual skill

- In sessions.csv, there are duplicate hashEmails (one user can have multiple sessions). This means that it could  be possible that observations are not independent of each other (same user -> similar behaviour)

## How Data Was Collected ##
- Player information collected by self-report
- Unique hashedEmail assigned randomly for identification
- Played_hours obtained by recording player's total playtime


## Question ##

The question that will be answered is: **"From the players.csv dataset, can a player's age and playtime hours predict whether they will subscribe to a game-related newsletter?"** To answer this question, we will use a K-NN classifier algorithm, using players.csv. We can see that age and played_hours are the only two numerical variables. The K-NN classifier only accepts numerical variables as predictor variables, which is why I chose age and played hours as our predictors, and then the response variable as subscribe.

